# defining a database

Here we aim at defining a database of faces which are looking either at the left, to the centre or to the right of the screen. This will be used for further supervised learning.

For a full definition, see ``2018-03-20_LeCheapEyeTracker_build_database``


In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
np.set_printoptions(precision=2, suppress=True)
# define plots to be inserted interactively
import cv2

## a minimalistic stimulation within the notebook


Instead of using an external program to show stimuli, we will use a minimalist solution within the notebook. This will consist of chains of characters that we really display into segments:

In [2]:
targets = {}
s_target = '🔴'
s_fixat = '🔵'
s_distra = '🌫'
s_blink = '☠️'

N_dis = 36 # <<<<<<<<<<<< change to adapt to your notebook's width
N_margin = 2

targets['left'] = N_margin * s_distra + s_target + 2 * N_dis * s_distra + N_margin * s_distra

targets['blink'] = N_margin * s_distra + N_dis * s_distra + s_blink + N_dis * s_distra + N_margin * s_distra

targets['center'] = N_margin * s_distra + N_dis * s_distra + s_target + N_dis * s_distra + N_margin * s_distra

targets['right'] = N_margin * s_distra + 2 * N_dis * s_distra + s_target + N_margin * s_distra

print ('Empty line')
null = N_margin * s_distra +  (2 * N_dis + 1) * s_distra + N_margin * s_distra
print (null)
print ('Fixation dot')
center = N_margin * s_distra + N_dis * s_distra + s_fixat + N_dis * s_distra + N_margin * s_distra
print (center)

for label in targets.keys():
    print('label=', label)
    print (targets[label])

Empty line
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
Fixation dot
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🔵🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
label= left
🌫🌫🔴🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
label= blink
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫☠️🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
label= center
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🔴🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
label= right
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🔴🌫🌫


## wrapping things up

All the protocol in one script:

In [3]:
import os
datapath = 'dataset'
try:
    os.mkdir(datapath)
except:
    pass
for target in targets.keys():
    print(os.path.join(datapath, target))
    try:
        os.mkdir(os.path.join(datapath, target))
    except:
        pass

import imageio
from LeCheapEyeTracker.EyeTrackerServer import Server
from IPython.display import clear_output

N_category = len(targets.keys())
def get_stack(datapath=datapath, subject_name='anonymous', N_frame=256, startup_time=1., interframe_time=.1, waiting_time=1.2):
    try:
        import time
        time.sleep(startup_time)
        et = Server()
        print ('Time', et.clock())
        
        bootstraped, max_trials, no_trials = False, 100, 0
        while not bootstraped:
            try:
                img0 = et.cam.grab()
                #img_face, res, t0 = et.process_frame(img0, et.clock())
                bootstraped = True
            except:
                if no_trials>max_trials:
                    raise('Failed!')
                else:
                    pass
            no_trials += 1

        #H, W, three = img_face.shape
        for i in range(N_frame):            
            clear_output()
            try:
                # presentation of stimulus
                i_choice = np.random.randint(N_category)
                label = list(targets.keys())[i_choice]
                print(null)
                print(null)
                print(targets[label])
                print(null)
                print(null)

                time.sleep(waiting_time)
                frame = et.cam.grab()
                #img_face, res, t0 = et.process_frame(frame, et.clock())

                clear_output()
                print(null)
                print(null)
                print(center)
                print(null)
                print(null)

                time.sleep(interframe_time)

                filename = os.path.join(datapath, label, subject_name + '_' + '%.3d' % i + '.png')
                imageio.imwrite(filename, frame[:, :, ::-1]) # converting on the fly from BGR > RGB
            except Exception as e:
                print('Failed for trial ', i, 'with message', e)
    finally:
        et.close()

get_stack(subject_name='2018-09-05_Laurent')

🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🔴🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
Capture released


KeyboardInterrupt: 

In [ ]:
#!ls -lR {datapath}

# detecting faces

In [8]:
import glob

In [9]:
if False:
    for target in targets.keys():
        for filename in glob.glob(os.path.join(datapath, target) + '/*.png'):
            frame = imageio.imread(filename)
            print(filename)

In [10]:
import os
datapath_faces = 'dataset_faces'
try:
    os.mkdir(datapath_faces)
except:
    pass
for target in targets.keys():
    print(os.path.join(datapath_faces, target))
    try:
        os.mkdir(os.path.join(datapath_faces, target))
    except:
        pass


dataset_faces/left
dataset_faces/blink
dataset_faces/center
dataset_faces/right


In [12]:
import dlib
detector = dlib.get_frontal_face_detector()

for target in targets.keys():
    for filename in glob.glob(os.path.join(datapath, target) + '/*.png'):
        frame = imageio.imread(filename)
        N_X, N_Y, three = frame.shape
        dets = detector(frame, 1)
        if False:
            print("Number of faces detected: {}".format(len(dets)))
            for i, d in enumerate(dets):
                print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
                    i, d.left(), d.top(), d.right(), d.bottom()))
                
        d = dets[0]
        if False:
            # Create figure and axes
            fig, ax = plt.subplots(figsize=(15, 8))

            # Display the image
            ax.imshow(frame)

            # Create a Rectangle patch
            rect = patches.Rectangle((d.bottom(), d.left()), d.right()-d.left(), d.top()-d.bottom(), linewidth=1, edgecolor='r', facecolor='none')

            # Add the patch to the Axes
            ax.add_patch(rect)
            plt.show()    
        if False:
            # Create figure and axes
            fig, ax = plt.subplots(figsize=(15, 8))

            # Display the cropped image
            ax.imshow(frame[(d.top()):(d.bottom()), (d.left()):(d.right()), :])

            plt.show()                
            
        filename_face = filename.replace(datapath, datapath_faces)
        imageio.imwrite(filename_face, frame[(d.top()):(d.bottom()), (d.left()):(d.right()), :]) 
